In [4]:
import matplotlib.pyplot as plt
import cv2
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, BatchNormalization, Dropout, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix
import seaborn as sns
import os

# Constants
IMG_DIM = 224
CHANNEL_SIZE = 3
NUM_CLASSES = 5
BATCH_SIZE = 32
EPOCHS = 50
SEED = 42

# Path to data
data_dir = '/home/gcekcse/Documents/ML_Project_hk/aptos2019-blindness-detection/G1/G1_images'
labels_file = '/home/gcekcse/Documents/ML_Project_hk/aptos2019-blindness-detection/G1/G1.csv'

# Load labels
df = pd.read_csv(labels_file)

# Split data into training and test sets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=SEED)

# Preprocessing function
def preprocess_image(image_path):
    img = cv2.imread(image_path)
    if img is None:
        print(f"Error loading {image_path}")
        return None
    img = cv2.resize(img, (IMG_DIM, IMG_DIM))
    img = img / 255.0  # Normalize
    return img

def preprocess_data(dataframe, data_dir):
    X = []
    y = []
    for idx, row in dataframe.iterrows():
        img_path = os.path.join(data_dir, row['id_code'] + ".png")
        img = preprocess_image(img_path)
        if img is not None:
            X.append(img)
            y.append(row['diagnosis'])
        if idx % 100 == 0:
            print(f"Processed {idx + 1}/{len(dataframe)} images.")
    return np.array(X), np.array(y)

# Preprocess training and test data
print("Preprocessing training data...")
X_train, y_train = preprocess_data(train_df, data_dir)
print("Preprocessing test data...")
X_test, y_test = preprocess_data(test_df, data_dir)

# Create data augmentation generator
datagen = ImageDataGenerator(rotation_range=20, zoom_range=0.15, width_shift_range=0.2,
                             height_shift_range=0.2, shear_range=0.15, horizontal_flip=True, fill_mode="nearest")



Preprocessing training data...
Processed 2901/2929 images.
Processed 2401/2929 images.
Processed 3401/2929 images.
Processed 3201/2929 images.
Processed 901/2929 images.
Processed 101/2929 images.
Processed 3101/2929 images.
Processed 1601/2929 images.
Processed 1101/2929 images.
Processed 1001/2929 images.
Processed 3001/2929 images.
Processed 301/2929 images.
Processed 2501/2929 images.
Processed 801/2929 images.
Processed 501/2929 images.
Processed 2701/2929 images.
Processed 3301/2929 images.
Processed 3501/2929 images.
Processed 2801/2929 images.
Processed 1701/2929 images.
Processed 1901/2929 images.
Processed 3601/2929 images.
Processed 2101/2929 images.
Processed 2001/2929 images.
Processed 1301/2929 images.
Processed 2201/2929 images.
Processed 401/2929 images.
Processed 1401/2929 images.
Processed 201/2929 images.
Processed 1501/2929 images.
Processed 601/2929 images.
Processed 2301/2929 images.
Preprocessing test data...
Processed 2601/733 images.
Processed 1801/733 images.


In [5]:
# Create ResNet50 model
def create_resnet(img_dim, CHANNEL, n_class):
    input_tensor = Input(shape=(img_dim, img_dim, CHANNEL))
    base_model = ResNet50(weights='imagenet', include_top=False, input_tensor=input_tensor)

    x = GlobalAveragePooling2D()(base_model.output)
    x = BatchNormalization()(x)
    x = Dropout(0.4)(x)
    x = Dense(2048, activation='elu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.4)(x)
    x = Dense(1024, activation='elu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)
    x = Dense(512, activation='elu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    output_layer = Dense(n_class, activation='softmax')(x)

    model_resnet = Model(input_tensor, output_layer)
    return model_resnet



2024-09-30 15:18:44.126178: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-30 15:18:44.231747: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-30 15:18:44.234698: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-12.2/lib64
2024-09-30 15:18:44.234716: I tensorflow/compiler/xla/st

In [6]:
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard
import datetime

# Create and compile the model
model_resnet = create_resnet(IMG_DIM, CHANNEL_SIZE, NUM_CLASSES)
model_resnet.compile(optimizer=Adam(learning_rate=1e-4), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Define ModelCheckpoint callback to save the model for each epoch
checkpoint_callback = ModelCheckpoint(
    filepath='model_checkpoint_epoch_{epoch:02d}.h5',  # Save model as 'model_checkpoint_epoch_xx.h5'
    save_freq='epoch',  # Save at the end of each epoch
    save_best_only=False,  # Set to True to only save the best model based on `monitor`
    save_weights_only=False,  # Save the full model (architecture + weights)
    verbose=1  # Print message after each save
)

# Define TensorBoard callback to log metrics
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(
    log_dir=log_dir,  # Specify the log directory
    histogram_freq=1,  # Record activation histograms every epoch
    write_graph=True,  # Visualize the graph
    write_images=True  # Log model weights as images
)

# Train the model with both ModelCheckpoint and TensorBoard callbacks
history = model_resnet.fit(
    datagen.flow(X_train, y_train, batch_size=BATCH_SIZE),
    validation_data=(X_test, y_test),
    epochs=EPOCHS,
    callbacks=[checkpoint_callback, tensorboard_callback]  # Add both callbacks
)


2024-09-30 15:23:46.348692: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


ValueError: The filepath provided must end in `.keras` (Keras model format). Received: filepath=model_checkpoint_epoch_{epoch:02d}.h5

In [ ]:

# Evaluate the model
(eval_loss, eval_accuracy) = model_resnet.evaluate(X_test, y_test)
print(f"Accuracy: {eval_accuracy * 100:.2f}%")
print(f"Loss: {eval_loss}")

# Classification report
y_pred = np.argmax(model_resnet.predict(X_test), axis=1)
print(classification_report(y_test, y_pred))

# ROC-AUC score
y_pred_prob = model_resnet.predict(X_test)
print(f"AUC-ROC: {roc_auc_score(y_test, y_pred_prob, multi_class='ovo')}")

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=np.unique(y_test), yticklabels=np.unique(y_test))
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.show()
